In [283]:
from math import sqrt
from scipy.optimize import minimize
from numpy.random import rand

## Probability parameters

Set parameters for probability function using $p(x) = \dfrac{x^a}{b + x^a}$ for $a \in [0,1]$ and $b > 0$.

In [284]:
a = 1/2
b = 1/5

Next line creates probability function and $g(x) = \dfrac{p(x)}{p'(x)} = \dfrac{x(x^a + b)}{ab}$

In [285]:
assert a >= 0 and a <= 1, 'Set a in [0,1]'
assert b > 0, 'Set b > 0'
p = lambda x: x**a / (b + x**a)
g = lambda x: x * (x**a + b) / (a * b)

## Initiator-optimal alpha-rule

Consider first the problem of finding the initiator-optimal alpha-rule. Specifically, each agent $i > 0$ will invest $x_i = c$ such that $p(c) \alpha = g(c)$. The initiator's expected payoff is
$$
(1 - p(x_0)) \cdot 1 + p(x_0) (1 - p(c)) \cdot 2 + p(x_0) p(c) (1 - p(c)) (2 + 1 - \alpha) + p(x_0) p^2(c) (1 - p(c)) (2 + 2(1 - \alpha)) + \dots - x_0
$$
This simplifies to 
$$
(1 - p(x_0)) + 2 p(x_0) (1 - p(c)) (1 + p(c) + p^2(c) + \dots) + p(x_0) p(c) (1 - p(c)) (1 - \alpha) (1 + 2p(c) + 3p^2(c) + \dots) - x_0,
$$
which becomes
$$
1 - p(x_0) + 2 p(x_0) + \dfrac{p(x_0) p(c) (1 - \alpha)}{1 - p(c)} - x_0.
$$
Simplifying further,
$$
1 + p(x_0) \left(1 + \dfrac{p(c) (1 - \alpha)}{1 - p(c)} \right) - x_0.
$$
which is
$$
1 + p(x_0) \left(\dfrac{1 - p(c) + p(c) - p(c) \alpha}{1 - p(c)} \right) - x_0,
$$
so
$$
1 + p(x_0) \left(\dfrac{1 - p(c) \alpha}{1 - p(c)} \right) - x_0.
$$
We incorporate the FOC for agents $i > 0$, namely $p(c) \alpha = g(c)$, and maximize the resulting expression with respect to $x_0$ and $c$:
$$
1 + p(x_0) \left( \dfrac{1 - g(c)}{1 - p(c)} \right ) - x_0.
$$
In particular, x[0] is $x_0$ while x[1] is $c$.
Instead of maximizing the function, we minimize the negative function.
We add also an initial guess x0 and non-negativity bounds.

In [286]:
initFun = lambda x: - (1 + p(x[0]) * (1 - g(x[1])) / (1 - p(x[1])) - x[0])
x0 = [1, 1]
bnds = [(0, None), (0, None)]
f = minimize(fun = initFun, x0 = x0, bounds = bnds)
# f.x is the solution, namely the investments [x0, c]
# f.fun is the (negative) function value at the optimum
c = f.x[1]
alpha = g(c) / p(c)

print(f)
print('-------------------------------------------------------------------------')
print('Init. investment x0:\t', f.x[0])
print('Other investment c:\t', c)
print('Init. prob. p(x0):\t', p(f.x[0]))
print('Other prob. p(c):\t', p(c))
print('Alpha:\t\t\t', alpha)

      fun: -1.9620455579400082
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([6.1284311e-06, 8.4865448e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 42
      nit: 11
     njev: 14
   status: 0
  success: True
        x: array([0.18248815, 0.04153322])
-------------------------------------------------------------------------
Init. investment x0:	 0.1824881483715041
Other investment c:	 0.04153321776256865
Init. prob. p(x0):	 0.681115491561614
Other prob. p(c):	 0.5047016215392468
Alpha:			 0.3322951239611897


## Two-optimality with alpha-rule

Assume now that the rule starts
\begin{align*}
& (1), \\
& (2, 0), \\
& (A, B, 3-A-B), \\
& (C, D, 4-C-D, 0), \\
& (C, D, 4-C-D + 1-\alpha, \alpha, 0), \\
& (C, D, 4-C-D + 2(1-\alpha), \alpha, \alpha, 0), \\
& \dots
\end{align*}
where $\alpha$ is as for the initiator-optimal alpha-rule above.
Therefore, we will have $x_3 = x_4 = \dots = c$ as above.

### Agent 0's FOC

For agent $0$, the expected return if successful $R_0$ is
\begin{align*}
R_0 &= (1 - p(x_1)) \cdot 2 + p(x_1) (1 - p(x_2)) \cdot A + p(x_1) p(x_2) (1 + p(c) + p^2(c) + \dots) (1 - p(c)) \cdot C \\
&= (1 - p(x_1)) \cdot 2 + p(x_1) (1 - p(x_2)) \cdot A + p(x_1) p(x_2) \cdot C \\
&= 2 - 2p(x_1) + A p(x_1) - A p(x_1) p(x_2) + C p(x_1) p(x_2) \\
&= 2 + (A - 2) p(x_1) + (C - A) p(x_1) p(x_2).
\end{align*}

Agent $0$'s FOC is $p(x_0) (R_0 - 1) = g(x_0)$, so
$$
1 + (A-2) p(x_1) + (C - A) p(x_1) p(x_2) = \frac{g(x_0)}{p(x_0)}.
$$

### Agent 1's FOC

For agent $1$, the expected return if successful $R_1$ is
\begin{align*}
R_1 &= (1 - p(x_2)) \cdot B + p(x_2) (1 + p(c) + p^2(c) + \dots) (1 - p(c)) \cdot D \\
&= (1 - p(x_2)) \cdot B + p(x_2) \cdot D \\
&= B + (D - B) p(x_2)
\end{align*}

Agent $1$'s FOC is $p(x_1) (R_1 - 0) = g(x_1)$, so
$$
B + (D - B) p(x_2) = \frac{g(x_1)}{p(x_1)}.
$$

### Agent 2's expected payoff

Finally, we turn to agent $2$'s expected payoff:
$$
p(x_0) p(x_1) (1 - p(x_2)) (3 - A - B) + p(x_0) p(x_1) p(x_2) (1 + p(c) + p^2(c) + \dots) (1 - p(c)) (4 - C - D) + p(x_0) p(x_1) p(x_2) (p(c) + 2p^2(c) + \dots) (1 - p(c)) (1 - \alpha) - p(x_0) p(x_1) x_2
$$
This simplifies to
$$
p(x_0) p(x_1) (1 - p(x_2)) (3 - A - B) + p(x_0) p(x_1) p(x_2) (4 - C - D) + \dfrac{p(x_0) p(x_1) p(x_2) p(c) (1 - \alpha)}{1 - p(c)} - p(x_0) p(x_1) x_2
$$
Rearranging,
$$
p(x_0) p(x_1) (3 - A - B - x_2) + p(x_0) p(x_1) p(x_2) (1 + A + B - C - D) + \dfrac{p(x_0) p(x_1) p(x_2) p(c) (1 - \alpha)}{1 - p(c)}
$$
Define $Z = \dfrac{p(c) (1 - \alpha)}{1 - p(c)}$.
Note that this only depends on the already computed values.
Then
$$
p(x_0) p(x_1) (3 - A - B - x_2) + p(x_0) p(x_1) p(x_2) (1 + A + B - C - D + Z).
$$

In [287]:
Z = p(c) * (1 - alpha) / (1 - p(c))
print(Z)

0.6803812576445818


### Constrained optimization problem 

We will choose $x_0$, $x_1$, $x_2$, $A$, $B$, $C$, and $D$ to maximize agent $2$'s expected payoff.
The list x = [$x_0, x_1, x_2, A, B, C, D$] will represent the solution.
Hence, x[3] refers to the payoff $A$.
Again, we minimize the negative function and impose non-negativity bounds.

In [288]:
fun = lambda x: - (p(x[0]) * p(x[1]) * (3 - x[3] - x[4] - x[2]) + p(x[0]) * p(x[1]) * p(x[2]) * (1 + x[3] + x[4] - x[5] - x[6] + Z))
x0 = [1, 1, 1, 1, 1, 1, 1]
bnds = [(0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)]

Next, we add constraints. First, $A + B \leq 3$ and $C + D \leq 4$. We indicate that it's an inequality and then the function that should be non-negative. Hence, the middle row captures $3 - A - B >= 0$.

In [289]:
cons = []
cons.append({'type': 'ineq', 'fun': lambda x: 3 - x[3] - x[4]})
cons.append({'type': 'ineq', 'fun': lambda x: 4 - x[5] - x[6]})

Next, we turn to the equilibrium constraints. We compute the FOCs for agents $0$ and $1$ above.
For these equality constraints, we provide the function that should equal zero.
\begin{align*}
1 + (A-2) p(x_1) + (C - A) p(x_1) p(x_2) &= \frac{g(x_0)}{p(x_0)} \\
B + (D - B) p(x_2) &= \frac{g(x_1)}{p(x_1)}.
\end{align*}


In [290]:
cons.append({'type': 'eq', 'fun': lambda x: 1 + (x[3] - 2) * p(x[1]) + (x[5] - x[3]) * p(x[1]) * p(x[2]) - g(x[0]) / p(x[0])})
cons.append({'type': 'eq', 'fun': lambda x: x[4] + (x[6] - x[4]) * p(x[2]) - g(x[1]) / p(x[1])})

In [291]:
f = minimize(fun = fun, x0 = x0, bounds = bnds, constraints = cons)
# f.x is the solution, namely [x0, x1, x2, A, B, C, D]
# f.fun is the (negative) function value at the optimum
print(f)
print('-------------------------------------------------------------------------')
print('Init. investment x0:\t', f.x[0])
print('Investment x1:\t\t', f.x[1])
print('Investment x2:\t\t', f.x[2])
print('Investment c:\t\t', c)
print('Init. prob. p(x0):\t', p(f.x[0]))
print('Prob. p(x1):\t\t', p(f.x[1]))
print('Prob. p(x2):\t\t', p(f.x[2]))
print('Prob. p(c):\t\t', p(c))
print('A:\t\t\t',f.x[3])
print('B:\t\t\t',f.x[4])
print('C:\t\t\t',f.x[5])
print('D:\t\t\t',f.x[6])

     fun: -0.7537757561089674
     jac: array([-4.15311622, -3.88114308, -0.10488512,  0.08483554,  0.08483554,
        0.18134051,  0.18134051])
 message: 'Optimization terminated successfully'
    nfev: 128
     nit: 15
    njev: 15
  status: 0
 success: True
       x: array([0.04423322, 0.04668002, 0.1827654 , 0.91698802, 0.66716836,
       0.68094744, 0.23684555])
-------------------------------------------------------------------------
Init. investment x0:	 0.044233219289761795
Investment x1:		 0.04668002342879334
Investment x2:		 0.18276540338870984
Investment c:		 0.04153321776256865
Init. prob. p(x0):	 0.512571927154983
Prob. p(x1):		 0.5192950203418613
Prob. p(x2):		 0.6812803379761472
Prob. p(c):		 0.5047016215392468
A:			 0.9169880182822858
B:			 0.6671683560981805
C:			 0.6809474417263375
D:			 0.23684555459582704


## Further constrained problem

The optimization problem may have multiple solutions.
If there is one with $C = D = 0$, then that solution is also a solution to the problem in which we maximize agent $2$'s expected payoff minus $C$ and $D$.
Hence, we can compare the optimal function values: if there is no difference, then this suggests that there exists a solution with $C = D = 0$.
(Again, due to minimizing, we add $C$ and $D$ instead.)

In [292]:
consFun = lambda x: fun(x) + x[5] + x[6]

In [293]:
g = minimize(fun = consFun, x0 = x0, bounds = bnds, constraints = cons)
# f.x is the solution, namely [x0, x1, x2, A, B, C, D]
# f.fun is the (negative) function value at the optimum
print(g)
print('-------------------------------------------------------------------------')
print('Init. investment x0:\t', g.x[0])
print('Investment x1:\t\t', g.x[1])
print('Investment x2:\t\t', g.x[2])
print('Investment c:\t\t', c)
print('Init. prob. p(x0):\t', p(g.x[0]))
print('Prob. p(x1):\t\t', p(g.x[1]))
print('Prob. p(x2):\t\t', p(g.x[2]))
print('Prob. p(c):\t\t', p(c))
print('A:\t\t\t',g.x[3])
print('B:\t\t\t',g.x[4])
print('C:\t\t\t',g.x[5])
print('D:\t\t\t',g.x[6])

     fun: -0.7514667953967952
     jac: array([-4.47079536, -4.46051801, -0.61203262,  0.08611748,  0.08611748,
        1.16894639,  1.16894638])
 message: 'Optimization terminated successfully'
    nfev: 215
     nit: 25
    njev: 25
  status: 0
 success: True
       x: array([4.16070667e-02, 4.16835874e-02, 1.53948362e-01, 2.01222282e+00,
       9.87777182e-01, 3.03603978e-16, 3.51963365e-16])
-------------------------------------------------------------------------
Init. investment x0:	 0.041607066734487906
Investment x1:		 0.04168358738235656
Investment x2:		 0.1539483624560772
Investment c:		 0.04153321776256865
Init. prob. p(x0):	 0.5049236623608333
Prob. p(x1):		 0.5051533186726063
Prob. p(x2):		 0.66236892672867
Prob. p(c):		 0.5047016215392468
A:			 2.0122228178041235
B:			 0.9877771821958801
C:			 3.036039780480494e-16
D:			 3.5196336540784315e-16


The important comparison is then the following:

In [294]:
print('Unconstrained:\t', f.fun)
print('Constrained:\t', g.fun)
print()
if (f.fun < g.fun + 1e-3 and f.fun > g.fun - 1e-3): print("Constraints don't affect optimal value, so there is a solution with C = D = 0")
else: print("Constraints DO affect optimal value, so there may NOT exist a solution with C = D = 0")

Unconstrained:	 -0.7537757561089674
Constrained:	 -0.7514667953967952

Constraints DO affect optimal value, so there may NOT exist a solution with C = D = 0
